In [1]:
import pandas as pd
import datetime

/var/folders/d8/5jnc6rls01s_1wrs9gvpkh_40000gn/T/ipykernel_86361/3813427381.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [24]:
df = pd.read_csv('.\contracts.csv')

In [76]:
excluded_types = ['10-Day Contract', 'Rookie Contract', 'Two-Way Contract', 'Exhibit 10', 'Camp Invite', 'Rookie Scale Contract']
df = df[~df.TYPE.isin(excluded_types)]

df.to_csv('contracts.csv', sep = ',' , decimal = '.', index = False)
df.head()

,PLAYER,AGE,POS,TEAM,DATE,TYPE,METHOD,EXTENSION,LENGTH,VALUE,AAV
0,Patty Mills,35,PG,MIA,"Mar 6, 2024",Veteran Contract,Minimum Salary Exception,0,1.0,"$753,186","$753,186"
1,Malik Shake Milton,27,SG,NYK,"Mar 5, 2024",Veteran Contract,Minimum Salary Exception,0,1.0,"$552,938","$552,938"
2,Kelly Olynyk,32,"C, PF",TOR,"Mar 4, 2024",Veteran Extension,Bird Exception,1,2.0,"$26,250,000","$13,125,000"
3,Kendall Brown,20,"SG, SF",IND,"Mar 3, 2024",Veteran Contract,Room-MLE,0,2.5,"$5,522,270","$1,840,757"
4,A.J. Lawson,23,SG,DAL,"Mar 3, 2024",Veteran Contract,MLE,0,4.0,"$7,912,022","$1,978,006"


In [78]:
options = df['LENGTH'].astype(str).str.contains('-').tolist()
out = []

for target in df.loc[options, 'LENGTH'].tolist():
    new_val = str(target).split('-')
    new_val = (int(new_val[0]) + int(new_val[-1])) / 2

    out.append(new_val)

df.loc[options, 'LENGTH'] = out
df.to_csv('contracts.csv', sep = ',' , decimal = '.', index = False)
df.head()

,PLAYER,AGE,POS,TEAM,DATE,TYPE,METHOD,EXTENSION,LENGTH,VALUE,AAV
0,Patty Mills,35,PG,MIA,"Mar 6, 2024",Veteran Contract,Minimum Salary Exception,0,1.0,"$753,186","$753,186"
1,Malik Shake Milton,27,SG,NYK,"Mar 5, 2024",Veteran Contract,Minimum Salary Exception,0,1.0,"$552,938","$552,938"
2,Kelly Olynyk,32,"C, PF",TOR,"Mar 4, 2024",Veteran Extension,Bird Exception,1,2.0,"$26,250,000","$13,125,000"
3,Kendall Brown,20,"SG, SF",IND,"Mar 3, 2024",Veteran Contract,Room-MLE,0,2.5,"$5,522,270","$1,840,757"
4,A.J. Lawson,23,SG,DAL,"Mar 3, 2024",Veteran Contract,MLE,0,4.0,"$7,912,022","$1,978,006"


In [25]:
point_gaurd = []
shooting_gaurd = []
small_forward = []
power_forward = []
center = []

positions = [('PG', point_gaurd), ('SG', shooting_gaurd), ('SF', small_forward), ('PF', power_forward), ('C', center)]

for i in df.index:
    for pos in positions:
        if pos[0] in df['POS'][i]:
            pos[1].append(1)

        else:
            pos[1].append(0)

for pos in positions:
    df.loc[:, pos[0]] = pos[1]

df = df.drop('POS', axis = 1)
df.head()

,PLAYER,AGE,TEAM,DATE,TYPE,METHOD,EXTENSION,LENGTH,VALUE,AAV,PG,SG,SF,PF,C
0,Patty Mills,35,MIA,"Mar 6, 2024",Veteran Contract,Minimum Salary Exception,0,1.0,"$753,186","$753,186",1,0,0,0,0
1,Malik Shake Milton,27,NYK,"Mar 5, 2024",Veteran Contract,Minimum Salary Exception,0,1.0,"$552,938","$552,938",0,1,0,0,0
2,Kelly Olynyk,32,TOR,"Mar 4, 2024",Veteran Extension,Bird Exception,1,2.0,"$26,250,000","$13,125,000",0,0,0,1,1
3,Kendall Brown,20,IND,"Mar 3, 2024",Veteran Contract,Room-MLE,0,2.5,"$5,522,270","$1,840,757",0,1,1,0,0
4,A.J. Lawson,23,DAL,"Mar 3, 2024",Veteran Contract,MLE,0,4.0,"$7,912,022","$1,978,006",0,1,0,0,0


In [26]:
for col in ['VALUE', 'AAV']:
    df[col] = df[col].str.replace('$', '').str.replace(',', '').astype(float)

df.head()

,PLAYER,AGE,TEAM,DATE,TYPE,METHOD,EXTENSION,LENGTH,VALUE,AAV,PG,SG,SF,PF,C
0,Patty Mills,35,MIA,"Mar 6, 2024",Veteran Contract,Minimum Salary Exception,0,1.0,753186.0,753186.0,1,0,0,0,0
1,Malik Shake Milton,27,NYK,"Mar 5, 2024",Veteran Contract,Minimum Salary Exception,0,1.0,552938.0,552938.0,0,1,0,0,0
2,Kelly Olynyk,32,TOR,"Mar 4, 2024",Veteran Extension,Bird Exception,1,2.0,26250000.0,13125000.0,0,0,0,1,1
3,Kendall Brown,20,IND,"Mar 3, 2024",Veteran Contract,Room-MLE,0,2.5,5522270.0,1840757.0,0,1,1,0,0
4,A.J. Lawson,23,DAL,"Mar 3, 2024",Veteran Contract,MLE,0,4.0,7912022.0,1978006.0,0,1,0,0,0


In [29]:
df['DATE'] = pd.to_datetime(df['DATE'])
df.to_csv('contracts.csv', sep = ',' , decimal = '.', index = False)
df.head()

,PLAYER,AGE,TEAM,DATE,TYPE,METHOD,EXTENSION,LENGTH,VALUE,AAV,PG,SG,SF,PF,C
0,Patty Mills,35,MIA,2024-03-06,Veteran Contract,Minimum Salary Exception,0,1.0,753186.0,753186.0,1,0,0,0,0
1,Malik Shake Milton,27,NYK,2024-03-05,Veteran Contract,Minimum Salary Exception,0,1.0,552938.0,552938.0,0,1,0,0,0
2,Kelly Olynyk,32,TOR,2024-03-04,Veteran Extension,Bird Exception,1,2.0,26250000.0,13125000.0,0,0,0,1,1
3,Kendall Brown,20,IND,2024-03-03,Veteran Contract,Room-MLE,0,2.5,5522270.0,1840757.0,0,1,1,0,0
4,A.J. Lawson,23,DAL,2024-03-03,Veteran Contract,MLE,0,4.0,7912022.0,1978006.0,0,1,0,0,0


In [ ]:
season_cutoffs = [[datetime.datetime(2023, 10, 24), datetime.datetime(2024, 4, 14)], 
                  [datetime.datetime(2022, 10, 18), datetime.datetime(2023, 4, 9)],
                  [datetime.datetime(2021, 10, 19), datetime.datetime(2022, 3, 10)]]
#find if mid season or off season
for date in df['DATE']:
    mid = False
    season = None
    for season_dates in season_cutoffs :
        if season_dates[0] <= date <= season_dates[1]:
            mid = True
            season = season_dates